## I practice Black-Litterman model based on the results got from APT. Here are the results from APT.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy import stats
from scipy.optimize import minimize
from scipy.optimize import NonlinearConstraint

### A. Calculate historical factor monthly returns for the following factors based on APT.

In [2]:
# Load data of Monthly Returns for S&P 500 Index and US Dollar Index.
df_SPD = pd.read_excel('APT&BLM.xlsx',sheet_name = 'SPX_DXY_MonthlyReturns',parse_dates=[0],index_col=[0])
df_SPD

,S&P - 500 Index,US Dollar Index
2001-07-31,-0.984416,-1.916799
2001-08-31,-6.260163,-3.208739
2001-09-30,-8.075231,-0.008817
2001-10-31,1.906875,1.278547
2001-11-30,7.670629,1.105694
...,...,...
2018-08-31,3.258530,0.619752
2018-09-30,0.569193,-0.008409
2018-10-31,-6.835016,2.097086
2018-11-30,2.037839,0.149289


In [3]:
df_500 = pd.read_excel('APT&BLM.xlsx',sheet_name = 'SP500Stocks',index_col=[0],skiprows=1)
df_500

,TICKER,SECTOR,Market Cap ($ Mil),2001-07-31 00:00:00,2001-08-31 00:00:00,2001-09-30 00:00:00,2001-10-31 00:00:00,2001-11-30 00:00:00,2001-12-31 00:00:00,2002-01-31 00:00:00,...,2018-03-31 00:00:00,2018-04-30 00:00:00,2018-05-31 00:00:00,2018-06-30 00:00:00,2018-07-31 00:00:00,2018-08-31 00:00:00,2018-09-30 00:00:00,2018-10-31 00:00:00,2018-11-30 00:00:00,2018-12-31 00:00:00
NAME,,,,,,,,,,,,,,,,,,,,,
3m Co,MMM,INDU,110949,-1.945100,-6.446533,-5.475902,6.077282,10.339559,3.168009,-6.098369,...,-6.789357,-11.447915,2.149656,-0.258595,7.930319,0.006810,-0.099670,-9.705245,10.030197,-8.358958
Abbott Labs,ABT,HLTH,127036,12.112976,-7.258951,4.326061,2.599966,3.812730,1.363484,3.886888,...,-0.679629,-2.525512,5.848923,-0.877646,7.939856,1.983455,9.754681,-5.645529,7.412211,-2.322711
Abbvie Inc,ABBV,HLTH,138674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-18.285435,3.070457,2.475368,-6.357350,0.546199,4.065904,-1.458604,-16.817462,21.091854,-2.206395
Abiomed Inc,ABMD,HLTH,14640,-19.423240,-1.315789,-6.826667,24.441900,-12.741490,-16.605166,-6.131479,...,8.505481,3.422798,26.645622,7.322769,-13.328444,14.681409,10.617837,-24.135631,-2.497069,-2.296501
Accenture Plc Ireland,ACN,INFT,89887,NaN,-0.401070,-14.429530,37.803922,28.628344,19.115044,-4.234770,...,-4.664319,-0.615572,3.002638,5.040437,-2.604036,6.113071,0.668382,-6.540393,4.377624,-14.290059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Xylem Inc,XYL,INDU,11991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.137550,-5.226213,-3.145842,-4.289844,13.624241,-0.578087,5.216744,-17.628674,11.283937,-8.577682
Yum Brands Inc,YUM,DSCR,28707,4.214608,-6.842003,-7.977550,28.990256,-6.206153,3.688246,13.414138,...,4.607980,2.314109,-6.221202,-3.823932,1.367929,10.055781,4.626532,-0.549981,2.423112,-0.325260
Zimmer Hldgs Inc,ZBH,HLTH,21156,NaN,-4.895105,2.022059,11.387387,4.367519,-5.331680,6.516045,...,-5.993991,5.621732,-3.177918,0.152802,12.634576,-1.505699,6.538274,-13.600042,3.019571,-11.156387


In [4]:
Beta_mat1 = np.zeros(shape=(505,3))

In [5]:
# Run a regression using equity market factor and US Dollar factor. Get the Beta matrix.
for i in range(len(df_500)):
    df_stock = df_500.iloc[i]
    df_stockret = df_stock[3:].astype('float')
    df_stockret = df_stockret.to_frame()
    df_step1 = pd.concat([df_stockret,df_SPD],axis=1)
    df_step1.dropna(inplace=True)
    df_step1.columns = ['Stock','SP','USD']
    list1 = df_step1.columns.tolist()
    formula1 = str(list1[0])+' ~ '+str(list1[1])
    del list1[0:2]
    for n in list1:
        formula1 += '+'
        formula1 += str(n)

    result1 = smf.ols(formula1,data=df_step1).fit()
    Beta_mat1[i]=result1.params.values

In [6]:
pd.DataFrame(Beta_mat1)

,0,1,2
0,0.411085,0.897279,0.137059
1,0.635259,0.501031,-0.096830
2,0.662926,1.450096,-0.556901
3,1.770226,1.263956,0.059849
4,0.772685,1.152855,0.193686
...,...,...,...
500,0.539917,1.037699,-0.747278
501,1.101420,0.657877,-0.258958
502,0.431365,0.721118,-0.503893
503,-0.190218,1.136268,-0.084878


In [7]:
# Sector factors. 
sector = df_500.iloc[:,1]
sector

NAME
3m Co                    INDU
Abbott Labs              HLTH
Abbvie Inc               HLTH
Abiomed Inc              HLTH
Accenture Plc Ireland    INFT
                         ... 
Xylem Inc                INDU
Yum Brands Inc           DSCR
Zimmer Hldgs Inc         HLTH
Zions Bancorp            FINA
Zoetis Inc               HLTH
Name: SECTOR, Length: 505, dtype: object

In [8]:
# Leaving out the UTIL.
sector1 = pd.get_dummies(df_500, columns = ['SECTOR'])
sector1.drop('SECTOR_UTIL',axis=1,inplace=True)
sector1.iloc[:,-11:]

,2018-12-31 00:00:00,SECTOR_DSCR,SECTOR_ENER,SECTOR_FINA,SECTOR_HLTH,SECTOR_INDU,SECTOR_INFT,SECTOR_MATS,SECTOR_REAL,SECTOR_STPL,SECTOR_TCOM
NAME,,,,,,,,,,,
3m Co,-8.358958,0,0,0,0,1,0,0,0,0,0
Abbott Labs,-2.322711,0,0,0,1,0,0,0,0,0,0
Abbvie Inc,-2.206395,0,0,0,1,0,0,0,0,0,0
Abiomed Inc,-2.296501,0,0,0,1,0,0,0,0,0,0
Accenture Plc Ireland,-14.290059,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
Xylem Inc,-8.577682,0,0,0,0,1,0,0,0,0,0
Yum Brands Inc,-0.325260,1,0,0,0,0,0,0,0,0,0
Zimmer Hldgs Inc,-11.156387,0,0,0,1,0,0,0,0,0,0


In [9]:
# Get sector exposures.
Beta_mat2 = Beta_mat1
sector_expo = sector1.iloc[:,-10:]
sector_expo

,SECTOR_DSCR,SECTOR_ENER,SECTOR_FINA,SECTOR_HLTH,SECTOR_INDU,SECTOR_INFT,SECTOR_MATS,SECTOR_REAL,SECTOR_STPL,SECTOR_TCOM
NAME,,,,,,,,,,
3m Co,0,0,0,0,1,0,0,0,0,0
Abbott Labs,0,0,0,1,0,0,0,0,0,0
Abbvie Inc,0,0,0,1,0,0,0,0,0,0
Abiomed Inc,0,0,0,1,0,0,0,0,0,0
Accenture Plc Ireland,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
Xylem Inc,0,0,0,0,1,0,0,0,0,0
Yum Brands Inc,1,0,0,0,0,0,0,0,0,0
Zimmer Hldgs Inc,0,0,0,1,0,0,0,0,0,0


In [10]:
# Put the sector exposures with Beta matrix 1 to get Beta matrix 2.
Beta_mat2 = pd.DataFrame(Beta_mat1,columns=['coef','beta_SP','beta_USD'],index=sector_expo.index)
Beta_mat2 = pd.concat([Beta_mat2,sector_expo],axis=1)
Beta_mat2 = Beta_mat2.iloc[:,1:]
Beta_mat2

,beta_SP,beta_USD,SECTOR_DSCR,SECTOR_ENER,SECTOR_FINA,SECTOR_HLTH,SECTOR_INDU,SECTOR_INFT,SECTOR_MATS,SECTOR_REAL,SECTOR_STPL,SECTOR_TCOM
NAME,,,,,,,,,,,,
3m Co,0.897279,0.137059,0,0,0,0,1,0,0,0,0,0
Abbott Labs,0.501031,-0.096830,0,0,0,1,0,0,0,0,0,0
Abbvie Inc,1.450096,-0.556901,0,0,0,1,0,0,0,0,0,0
Abiomed Inc,1.263956,0.059849,0,0,0,1,0,0,0,0,0,0
Accenture Plc Ireland,1.152855,0.193686,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
Xylem Inc,1.037699,-0.747278,0,0,0,0,1,0,0,0,0,0
Yum Brands Inc,0.657877,-0.258958,1,0,0,0,0,0,0,0,0,0
Zimmer Hldgs Inc,0.721118,-0.503893,0,0,0,1,0,0,0,0,0,0


In [11]:
factor_ret = np.zeros(shape=(13,210))

In [12]:
# Size factor
size_factor = df_500.iloc[:,[1,2]]
size_factor['Market Cap ($ Mil)'] = np.log(size_factor['Market Cap ($ Mil)'])
size_factor

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,SECTOR,Market Cap ($ Mil)
NAME,,
3m Co,INDU,11.616826
Abbott Labs,HLTH,11.752226
Abbvie Inc,HLTH,11.839881
Abiomed Inc,HLTH,9.591513
Accenture Plc Ireland,INFT,11.406309
...,...,...
Xylem Inc,INDU,9.391912
Yum Brands Inc,DSCR,10.264896
Zimmer Hldgs Inc,HLTH,9.959679


In [13]:
size_sec = size_factor.groupby('SECTOR').mean()
size_sec.columns = ['MarketCapMean']
size_sec

,MarketCapMean
SECTOR,
DSCR,9.668220
ENER,9.922564
FINA,10.069093
HLTH,10.382486
INDU,9.887828
INFT,10.225671
MATS,9.724490
REAL,9.682977
STPL,10.415517


In [14]:
size_sec['MarketCapStd'] = size_factor.groupby('SECTOR').std()
size_sec

,MarketCapMean,MarketCapStd
SECTOR,,
DSCR,9.668220,1.016778
ENER,9.922564,1.018311
FINA,10.069093,1.008906
HLTH,10.382486,1.014239
INDU,9.887828,0.874319
INFT,10.225671,1.136336
MATS,9.724490,0.759010
REAL,9.682977,0.629647
STPL,10.415517,0.985399


In [15]:
df_500_1 = df_500.copy()
for i in size_sec.index:
    df_500_1.loc[(df_500_1['SECTOR']==i),'MarketCapMean']=size_sec.loc[i,'MarketCapMean']
    df_500_1.loc[(df_500_1['SECTOR']==i),'MarketCapStd']=size_sec.loc[i,'MarketCapStd']

df_500_1

,TICKER,SECTOR,Market Cap ($ Mil),2001-07-31 00:00:00,2001-08-31 00:00:00,2001-09-30 00:00:00,2001-10-31 00:00:00,2001-11-30 00:00:00,2001-12-31 00:00:00,2002-01-31 00:00:00,...,2018-05-31 00:00:00,2018-06-30 00:00:00,2018-07-31 00:00:00,2018-08-31 00:00:00,2018-09-30 00:00:00,2018-10-31 00:00:00,2018-11-30 00:00:00,2018-12-31 00:00:00,MarketCapMean,MarketCapStd
NAME,,,,,,,,,,,,,,,,,,,,,
3m Co,MMM,INDU,110949,-1.945100,-6.446533,-5.475902,6.077282,10.339559,3.168009,-6.098369,...,2.149656,-0.258595,7.930319,0.006810,-0.099670,-9.705245,10.030197,-8.358958,9.887828,0.874319
Abbott Labs,ABT,HLTH,127036,12.112976,-7.258951,4.326061,2.599966,3.812730,1.363484,3.886888,...,5.848923,-0.877646,7.939856,1.983455,9.754681,-5.645529,7.412211,-2.322711,10.382486,1.014239
Abbvie Inc,ABBV,HLTH,138674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.475368,-6.357350,0.546199,4.065904,-1.458604,-16.817462,21.091854,-2.206395,10.382486,1.014239
Abiomed Inc,ABMD,HLTH,14640,-19.423240,-1.315789,-6.826667,24.441900,-12.741490,-16.605166,-6.131479,...,26.645622,7.322769,-13.328444,14.681409,10.617837,-24.135631,-2.497069,-2.296501,10.382486,1.014239
Accenture Plc Ireland,ACN,INFT,89887,NaN,-0.401070,-14.429530,37.803922,28.628344,19.115044,-4.234770,...,3.002638,5.040437,-2.604036,6.113071,0.668382,-6.540393,4.377624,-14.290059,10.225671,1.136336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Xylem Inc,XYL,INDU,11991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.145842,-4.289844,13.624241,-0.578087,5.216744,-17.628674,11.283937,-8.577682,9.887828,0.874319
Yum Brands Inc,YUM,DSCR,28707,4.214608,-6.842003,-7.977550,28.990256,-6.206153,3.688246,13.414138,...,-6.221202,-3.823932,1.367929,10.055781,4.626532,-0.549981,2.423112,-0.325260,9.668220,1.016778
Zimmer Hldgs Inc,ZBH,HLTH,21156,NaN,-4.895105,2.022059,11.387387,4.367519,-5.331680,6.516045,...,-3.177918,0.152802,12.634576,-1.505699,6.538274,-13.600042,3.019571,-11.156387,10.382486,1.014239


In [16]:
# Calculate z-score of log(mkt_cap).
df_500_1['zscore']=(np.log(df_500_1['Market Cap ($ Mil)'])-df_500_1['MarketCapMean'])/df_500_1['MarketCapStd']
df_500_1

,TICKER,SECTOR,Market Cap ($ Mil),2001-07-31 00:00:00,2001-08-31 00:00:00,2001-09-30 00:00:00,2001-10-31 00:00:00,2001-11-30 00:00:00,2001-12-31 00:00:00,2002-01-31 00:00:00,...,2018-06-30 00:00:00,2018-07-31 00:00:00,2018-08-31 00:00:00,2018-09-30 00:00:00,2018-10-31 00:00:00,2018-11-30 00:00:00,2018-12-31 00:00:00,MarketCapMean,MarketCapStd,zscore
NAME,,,,,,,,,,,,,,,,,,,,,
3m Co,MMM,INDU,110949,-1.945100,-6.446533,-5.475902,6.077282,10.339559,3.168009,-6.098369,...,-0.258595,7.930319,0.006810,-0.099670,-9.705245,10.030197,-8.358958,9.887828,0.874319,1.977538
Abbott Labs,ABT,HLTH,127036,12.112976,-7.258951,4.326061,2.599966,3.812730,1.363484,3.886888,...,-0.877646,7.939856,1.983455,9.754681,-5.645529,7.412211,-2.322711,10.382486,1.014239,1.350510
Abbvie Inc,ABBV,HLTH,138674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-6.357350,0.546199,4.065904,-1.458604,-16.817462,21.091854,-2.206395,10.382486,1.014239,1.436935
Abiomed Inc,ABMD,HLTH,14640,-19.423240,-1.315789,-6.826667,24.441900,-12.741490,-16.605166,-6.131479,...,7.322769,-13.328444,14.681409,10.617837,-24.135631,-2.497069,-2.296501,10.382486,1.014239,-0.779869
Accenture Plc Ireland,ACN,INFT,89887,NaN,-0.401070,-14.429530,37.803922,28.628344,19.115044,-4.234770,...,5.040437,-2.604036,6.113071,0.668382,-6.540393,4.377624,-14.290059,10.225671,1.136336,1.038986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Xylem Inc,XYL,INDU,11991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.289844,13.624241,-0.578087,5.216744,-17.628674,11.283937,-8.577682,9.887828,0.874319,-0.567203
Yum Brands Inc,YUM,DSCR,28707,4.214608,-6.842003,-7.977550,28.990256,-6.206153,3.688246,13.414138,...,-3.823932,1.367929,10.055781,4.626532,-0.549981,2.423112,-0.325260,9.668220,1.016778,0.586831
Zimmer Hldgs Inc,ZBH,HLTH,21156,NaN,-4.895105,2.022059,11.387387,4.367519,-5.331680,6.516045,...,0.152802,12.634576,-1.505699,6.538274,-13.600042,3.019571,-11.156387,10.382486,1.014239,-0.416871


In [17]:
# Put size factor with Beta Matrix to get the final Beta Matrix.
Beta_mat2 = pd.concat([Beta_mat2,df_500_1['zscore']],axis=1)
Beta_mat2

,beta_SP,beta_USD,SECTOR_DSCR,SECTOR_ENER,SECTOR_FINA,SECTOR_HLTH,SECTOR_INDU,SECTOR_INFT,SECTOR_MATS,SECTOR_REAL,SECTOR_STPL,SECTOR_TCOM,zscore
NAME,,,,,,,,,,,,,
3m Co,0.897279,0.137059,0,0,0,0,1,0,0,0,0,0,1.977538
Abbott Labs,0.501031,-0.096830,0,0,0,1,0,0,0,0,0,0,1.350510
Abbvie Inc,1.450096,-0.556901,0,0,0,1,0,0,0,0,0,0,1.436935
Abiomed Inc,1.263956,0.059849,0,0,0,1,0,0,0,0,0,0,-0.779869
Accenture Plc Ireland,1.152855,0.193686,0,0,0,0,0,1,0,0,0,0,1.038986
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Xylem Inc,1.037699,-0.747278,0,0,0,0,1,0,0,0,0,0,-0.567203
Yum Brands Inc,0.657877,-0.258958,1,0,0,0,0,0,0,0,0,0,0.586831
Zimmer Hldgs Inc,0.721118,-0.503893,0,0,0,1,0,0,0,0,0,0,-0.416871


In [18]:
# Run cross-sectional regression.
for i in range(210):
    reg = pd.concat([df_500.iloc[:,i+3],Beta_mat2],axis=1)
    reg.dropna(inplace=True)
    X = reg.iloc[:,-13:]
    X = sm.add_constant(X)
    Y = reg.iloc[:,0]

    result2 = sm.OLS(Y,X).fit()
    result2.summary()
    factor_ret[:,i] = result2.params.values[1:]
    
factor_ret

array([[ -6.86285728,  -6.53110722, -20.77960007, ...,  -7.03079982,
          0.49019469,  -3.17679038],
       [  1.14889942,  -3.81494125,  -4.42704275, ...,   3.36258877,
          1.69933723,   0.48735719],
       [ 12.19409328,  -0.84627951,   6.14513207, ...,  -4.72032772,
         -1.78038685,  -3.9030419 ],
       ...,
       [  6.4748542 ,   0.5861965 ,   6.04239303, ...,  -0.9762139 ,
         -2.04001118,  -6.06136948],
       [  4.82512414,  -0.83342392,   5.57295535, ...,  -0.7417938 ,
         -4.75218037,  -4.66125462],
       [ -0.94741375,  -0.93846446,  -0.14960713, ...,   0.48634149,
          0.58367353,   1.49874591]])

In [19]:
factor_ret = pd.DataFrame(factor_ret)
factor_ret

,0,1,2,3,4,5,6,7,8,9,...,200,201,202,203,204,205,206,207,208,209
0,-6.862857,-6.531107,-20.779600,9.164590,11.163984,1.215029,-4.323136,-5.533380,6.677938,-8.524798,...,-1.541033,-2.245750,2.079768,-2.869671,1.789404,-1.263999,-0.475853,-7.030800,0.490195,-3.176790
1,1.148899,-3.814941,-4.427043,5.534274,5.878303,-0.502738,-1.612277,-3.514096,-0.359773,-4.333348,...,-0.842876,0.077256,3.955822,1.263941,0.014721,1.749081,-1.058467,3.362589,1.699337,0.487357
2,12.194093,-0.846280,6.145132,0.473640,4.318661,3.370005,10.697396,8.667919,-11.619652,9.601583,...,-5.087975,0.796624,0.358355,1.071403,-1.414894,0.579908,1.160565,-4.720328,-1.780387,-3.903042
3,5.315452,-3.381388,4.087406,7.848457,-8.522760,3.799435,-1.851951,9.980194,-2.622675,1.177210,...,-0.680842,9.672145,3.682521,0.755149,-0.401137,-4.342493,2.554841,-10.515252,-7.983625,-8.856122
4,7.337022,-0.366846,17.953474,-9.750611,-0.611098,-1.179031,4.286246,7.722149,-9.511225,4.989156,...,-5.830681,-0.698454,-2.097977,-2.526363,0.872308,-0.221929,-1.441822,-2.727631,-0.899673,-5.496160
5,4.397612,0.190909,8.834765,2.879946,0.536815,-3.196261,0.527301,2.695108,-8.177159,-0.901076,...,-5.581573,-1.403980,0.652143,0.176648,3.401090,2.786344,3.402252,-8.664647,1.826115,-4.603050
6,9.781195,0.346426,9.998076,-3.630006,2.105712,0.565578,7.164433,10.661489,-10.552080,3.759445,...,-4.432178,-4.305730,1.022698,-2.894414,4.583613,0.885673,2.033739,-8.910813,1.470327,-5.440288
7,7.325754,-0.635126,8.745651,8.241545,3.268029,-1.688333,7.519178,0.528758,-6.466756,-2.971178,...,-4.275920,-0.903985,2.423637,-0.614386,-1.094338,5.066053,1.080130,-6.393504,-2.400866,-2.210402
8,9.080813,2.162124,10.472454,-1.789567,4.865412,-2.591446,6.720628,9.879490,-7.862185,0.769461,...,-6.761645,-0.783629,2.462243,-0.256819,-0.305439,-2.074810,0.625090,-5.299211,1.504166,-2.081822
9,3.769479,1.999704,12.708085,-6.354562,0.613596,-2.717193,1.122977,3.366614,-6.494473,3.682304,...,-0.057924,-1.057510,2.456788,3.748026,-1.893627,1.655226,-2.214345,-0.162410,2.529691,-2.285940


In [20]:
factor_ret_sectors = factor_ret.loc[2:11]
factor_ret_sectors

,0,1,2,3,4,5,6,7,8,9,...,200,201,202,203,204,205,206,207,208,209
2,12.194093,-0.846280,6.145132,0.473640,4.318661,3.370005,10.697396,8.667919,-11.619652,9.601583,...,-5.087975,0.796624,0.358355,1.071403,-1.414894,0.579908,1.160565,-4.720328,-1.780387,-3.903042
3,5.315452,-3.381388,4.087406,7.848457,-8.522760,3.799435,-1.851951,9.980194,-2.622675,1.177210,...,-0.680842,9.672145,3.682521,0.755149,-0.401137,-4.342493,2.554841,-10.515252,-7.983625,-8.856122
4,7.337022,-0.366846,17.953474,-9.750611,-0.611098,-1.179031,4.286246,7.722149,-9.511225,4.989156,...,-5.830681,-0.698454,-2.097977,-2.526363,0.872308,-0.221929,-1.441822,-2.727631,-0.899673,-5.496160
5,4.397612,0.190909,8.834765,2.879946,0.536815,-3.196261,0.527301,2.695108,-8.177159,-0.901076,...,-5.581573,-1.403980,0.652143,0.176648,3.401090,2.786344,3.402252,-8.664647,1.826115,-4.603050
6,9.781195,0.346426,9.998076,-3.630006,2.105712,0.565578,7.164433,10.661489,-10.552080,3.759445,...,-4.432178,-4.305730,1.022698,-2.894414,4.583613,0.885673,2.033739,-8.910813,1.470327,-5.440288
7,7.325754,-0.635126,8.745651,8.241545,3.268029,-1.688333,7.519178,0.528758,-6.466756,-2.971178,...,-4.275920,-0.903985,2.423637,-0.614386,-1.094338,5.066053,1.080130,-6.393504,-2.400866,-2.210402
8,9.080813,2.162124,10.472454,-1.789567,4.865412,-2.591446,6.720628,9.879490,-7.862185,0.769461,...,-6.761645,-0.783629,2.462243,-0.256819,-0.305439,-2.074810,0.625090,-5.299211,1.504166,-2.081822
9,3.769479,1.999704,12.708085,-6.354562,0.613596,-2.717193,1.122977,3.366614,-6.494473,3.682304,...,-0.057924,-1.057510,2.456788,3.748026,-1.893627,1.655226,-2.214345,-0.162410,2.529691,-2.285940
10,6.474854,0.586196,6.042393,-0.455695,2.977548,-3.028078,4.530565,6.709852,-9.232969,0.912770,...,-5.084995,-5.455916,-2.193325,2.916290,0.567354,-0.416350,0.067406,-0.976214,-2.040011,-6.061369
11,4.825124,-0.833424,5.572955,3.509324,0.349600,-2.472622,1.867889,9.126370,-9.292503,1.268775,...,-8.487334,-0.950837,1.476050,9.945045,-5.979237,0.840789,3.112886,-0.741794,-4.752180,-4.661255


### A. Without active views, given Pi (from 1) and Sigma (from 2), locate the optimal portfolio of the 10 sectors which has the highest expected return and at the same time its annualized volatility no greater than 10%.

In [21]:
# calculate Pi.
Pi = pd.DataFrame()
Pi['Mean'] = factor_ret_sectors.mean(axis=1)
Pi = Pi*0.01
Pi

,Mean
2,0.003106
3,0.001743
4,-0.001602
5,0.004106
6,0.002088
7,0.003271
8,0.002607
9,0.003634
10,0.001316
11,0.000973


In [22]:
# calculate covarience matrix.
Sigma = np.cov(factor_ret_sectors*0.01)
Sigma = pd.DataFrame(Sigma, index=Beta_mat2.columns[2:12], columns = Beta_mat2.columns[2:12])
Sigma

,SECTOR_DSCR,SECTOR_ENER,SECTOR_FINA,SECTOR_HLTH,SECTOR_INDU,SECTOR_INFT,SECTOR_MATS,SECTOR_REAL,SECTOR_STPL,SECTOR_TCOM
SECTOR_DSCR,0.002448,0.001020,0.001824,0.001399,0.001760,0.001408,0.001634,0.001263,0.001387,0.001555
SECTOR_ENER,0.001020,0.003621,0.001083,0.000897,0.001319,0.001309,0.001527,0.000225,0.000764,0.001325
SECTOR_FINA,0.001824,0.001083,0.002533,0.001455,0.001721,0.001271,0.001592,0.001146,0.001326,0.001420
SECTOR_HLTH,0.001399,0.000897,0.001455,0.001716,0.001342,0.001288,0.001240,0.000924,0.001105,0.001257
SECTOR_INDU,0.001760,0.001319,0.001721,0.001342,0.001902,0.001373,0.001662,0.000920,0.001218,0.001348
SECTOR_INFT,0.001408,0.001309,0.001271,0.001288,0.001373,0.001977,0.001443,0.000627,0.001035,0.001463
SECTOR_MATS,0.001634,0.001527,0.001592,0.001240,0.001662,0.001443,0.002004,0.000808,0.001118,0.001445
SECTOR_REAL,0.001263,0.000225,0.001146,0.000924,0.000920,0.000627,0.000808,0.002056,0.000798,0.000709
SECTOR_STPL,0.001387,0.000764,0.001326,0.001105,0.001218,0.001035,0.001118,0.000798,0.001296,0.001221
SECTOR_TCOM,0.001555,0.001325,0.001420,0.001257,0.001348,0.001463,0.001445,0.000709,0.001221,0.002027


In [23]:
# run an optimization.
pi = np.array(Pi)
sigma = np.array(Sigma)
sigma_mat = np.mat(sigma)
pi_mat = np.mat(pi)
def var(weights):
    return -np.sqrt(np.dot(weights.T,np.dot(sigma,weights)))
    #return -np.sqrt(weights.T*sigma_mat*weights)

def target(weights):
    return -np.sum(weights.T * pi)
cons = ({'type': 'ineq', 'fun': lambda x: var(x) + 0.1/np.sqrt(12)},{'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
x0 = np.array([[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]]).T
res = minimize(target, x0, method='SLSQP', constraints=cons)
res

     fun: -0.021242545924369968
     jac: array([-0.02124255, -0.02124255, -0.02124255, -0.02124255, -0.02124255,
       -0.02124255, -0.02124255, -0.02124255, -0.02124255, -0.02124255])
 message: 'Positive directional derivative for linesearch'
    nfev: 1205
     nit: 69
    njev: 65
  status: 8
 success: False
       x: array([-0.26503263,  0.13504025, -0.11360912,  0.07646105, -0.03044503,
        0.14754052,  0.07062962,  0.3379652 ,  0.68260172, -0.04115158])

In [24]:
var = np.dot(res.x.T,np.dot(sigma,res.x))
np.sqrt(var)

0.031112034439145047

In [25]:
np.sum(res.x)

0.9999999999999999

In [26]:
# the optimal portfolio of the 10 sectors.
pd.DataFrame(res.x,index=Beta_mat2.columns[2:12],columns=['MKT_opt'])

,MKT_opt
SECTOR_DSCR,-0.265033
SECTOR_ENER,0.135040
SECTOR_FINA,-0.113609
SECTOR_HLTH,0.076461
SECTOR_INDU,-0.030445
SECTOR_INFT,0.147541
SECTOR_MATS,0.070630
SECTOR_REAL,0.337965
SECTOR_STPL,0.682602
SECTOR_TCOM,-0.041152


### B. Now incorporate the active views on 4 using Black-Litterman model.

In [27]:
tau = 0.05

# view 1: ENER sector will have return 0.2% next month.
tmp_pos=1
port_v1=np.zeros((10,1))
port_v1[tmp_pos]=1
q1=0.002

# view 2: Sector FINA will outperform Sector INDU by 0.05% in next month.
tmpv2=2
tmpv22=4
port_v2=np.zeros((10,1))
port_v2[tmpv2]=1
port_v2[tmpv22]=-1
q2=0.0005

# view 3: Sector INFT and Sector HLTH will outperform Sector REAL and Sector STPL by 0.30% in next month.
tmp_long=[5,3]
tmp_short=[7,8]
port_v3_long=np.zeros((10,1))
port_v3_short=np.zeros((10,1))
port_v3_long[tmp_long]=0.5
port_v3_short[tmp_short]=-0.5
port_v3 = port_v3_long+port_v3_short 
q3=0.003

In [28]:
port_v1 = np.mat(port_v1).T

In [29]:
port_v2 = np.mat(port_v2).T

In [30]:
port_v3 = np.mat(port_v3).T

In [31]:
P = np.concatenate((port_v1,port_v2,port_v3),axis=0)
P

matrix([[ 0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  1. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0.5,  0. ,  0.5,  0. , -0.5, -0.5,  0. ]])

In [32]:
Q = np.array([q1,q2,q3])
Q = np.mat(Q)
Q = Q.T
Q

matrix([[0.002 ],
        [0.0005],
        [0.003 ]])

In [33]:
sigma_mat = np.mat(sigma)
# calculate the diagonal elements (corresponding to View 1,2,3)
omega1 = tau*(port_v1*sigma_mat*port_v1.T)
omega2 = tau*(port_v2*sigma_mat*port_v2.T)
omega3 = tau*(port_v3*sigma_mat*port_v3.T)

# the view uncertainty matrix
ome = np.array((omega1[0,0],omega2[0,0],omega3[0,0]))
Omega = np.mat(np.diag(ome))
Omega

matrix([[1.81044112e-04, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 4.96546067e-05, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 4.79269266e-05]])

### 1. I show the updated factor expected return distribution and compare it with the original one, i.e., calculate nu, which is the update return vector; and (H^-1+Sigma), which is the updated covariance matrix.

In [34]:
pi_mat = np.mat(pi)
# calculate C
C = tau * sigma_mat

# calculate H
H = P.T*Omega.I*P+C.I

# calculate nu
nu = H.I*(P.T*Omega.I*Q+C.I*pi_mat)

expr = np.concatenate((pi,nu),axis=1)
pd.DataFrame(expr,columns=['Pi','nu'],index=Beta_mat2.columns[2:12])

,Pi,nu
SECTOR_DSCR,0.003106,0.003347
SECTOR_ENER,0.001743,0.001951
SECTOR_FINA,-0.001602,0.000342
SECTOR_HLTH,0.004106,0.004909
SECTOR_INDU,0.002088,0.002028
SECTOR_INFT,0.003271,0.003957
SECTOR_MATS,0.002607,0.002894
SECTOR_REAL,0.003634,0.003410
SECTOR_STPL,0.001316,0.001589
SECTOR_TCOM,0.000973,0.001588


In [35]:
# updated covariance matrix
nu = np.array(nu)
update_sigma = H.I+sigma_mat
update_sigma = np.array(update_sigma)
pd.DataFrame(update_sigma,index=Beta_mat2.columns[2:12],columns = Beta_mat2.columns[2:12])

,SECTOR_DSCR,SECTOR_ENER,SECTOR_FINA,SECTOR_HLTH,SECTOR_INDU,SECTOR_INFT,SECTOR_MATS,SECTOR_REAL,SECTOR_STPL,SECTOR_TCOM
SECTOR_DSCR,0.002563,0.001046,0.001905,0.001462,0.001839,0.001469,0.001705,0.001324,0.001451,0.001623
SECTOR_ENER,0.001046,0.003709,0.001112,0.000916,0.001350,0.001336,0.001563,0.000237,0.000784,0.001356
SECTOR_FINA,0.001905,0.001112,0.002634,0.001517,0.001800,0.001324,0.001660,0.001198,0.001384,0.001478
SECTOR_HLTH,0.001462,0.000916,0.001517,0.001791,0.001400,0.001337,0.001289,0.000976,0.001155,0.001308
SECTOR_INDU,0.001839,0.001350,0.001800,0.001400,0.001984,0.001426,0.001730,0.000968,0.001273,0.001403
SECTOR_INFT,0.001469,0.001336,0.001324,0.001337,0.001426,0.002051,0.001497,0.000669,0.001081,0.001519
SECTOR_MATS,0.001705,0.001563,0.001660,0.001289,0.001730,0.001497,0.002086,0.000851,0.001166,0.001501
SECTOR_REAL,0.001324,0.000237,0.001198,0.000976,0.000968,0.000669,0.000851,0.002145,0.000835,0.000747
SECTOR_STPL,0.001451,0.000784,0.001384,0.001155,0.001273,0.001081,0.001166,0.000835,0.001356,0.001275
SECTOR_TCOM,0.001623,0.001356,0.001478,0.001308,0.001403,0.001519,0.001501,0.000747,0.001275,0.002113


### 2. Make observations on how active views impact updated returns.

Observations: Active views impact the updated returns that the returns of Sector DSCR, ENER, FINA, HLTH, INFT, MATS, STPL, TCOM increase and the returns of Sector INDU, REAL decrease.

### 3. Re-run the optimization as in A, but with the new return vector and covariance matrix. Compare the updated optimal portfolio (the weight vector) with the solution from A.

In [36]:
# the optimal weight vector from BL model.2. make observations on how active views impact updated returns.

def target1(weights):
    return -np.sum(weights * nu)
def vari(weights):
    return -np.sqrt(np.dot(weights.T,np.dot(update_sigma,weights)))
cons1 = ({'type': 'ineq', 'fun': lambda x: vari(x) + 0.1/np.sqrt(12)},{'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
x0 = np.array([[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]]).T
res1 = minimize(target1, x0, method='SLSQP', constraints=cons1)
res1

     fun: -0.026014951863526172
     jac: array([-0.02601495, -0.02601495, -0.02601495, -0.02601495, -0.02601495,
       -0.02601495, -0.02601495, -0.02601495, -0.02601495, -0.02601495])
 message: 'Positive directional derivative for linesearch'
    nfev: 1370
     nit: 79
    njev: 75
  status: 8
 success: False
       x: array([-0.26329773,  0.14079341, -0.11223828,  0.07493027, -0.0308344 ,
        0.14555009,  0.07016923,  0.3367235 ,  0.67906786, -0.04086395])

In [37]:
var = np.dot(res.x.T,np.dot(sigma,res.x))
print(np.sqrt(var))
print(np.sum(res.x))

0.031112034439145047
0.9999999999999999


In [38]:
w_mkt = res.x
w_mkt = np.mat(w_mkt).T
w_mkt

matrix([[-0.26503263],
        [ 0.13504025],
        [-0.11360912],
        [ 0.07646105],
        [-0.03044503],
        [ 0.14754052],
        [ 0.07062962],
        [ 0.3379652 ],
        [ 0.68260172],
        [-0.04115158]])

In [39]:
new_w_mkt = np.mat(res1.x).T
new_w_mkt

matrix([[-0.26329773],
        [ 0.14079341],
        [-0.11223828],
        [ 0.07493027],
        [-0.0308344 ],
        [ 0.14555009],
        [ 0.07016923],
        [ 0.3367235 ],
        [ 0.67906786],
        [-0.04086395]])

In [40]:
weightverse = np.concatenate((w_mkt,new_w_mkt),axis=1)
pd.DataFrame(weightverse,columns=['MKT_opt','BL_opt'],index=Beta_mat2.columns[2:12])

,MKT_opt,BL_opt
SECTOR_DSCR,-0.265033,-0.263298
SECTOR_ENER,0.135040,0.140793
SECTOR_FINA,-0.113609,-0.112238
SECTOR_HLTH,0.076461,0.074930
SECTOR_INDU,-0.030445,-0.030834
SECTOR_INFT,0.147541,0.145550
SECTOR_MATS,0.070630,0.070169
SECTOR_REAL,0.337965,0.336724
SECTOR_STPL,0.682602,0.679068
SECTOR_TCOM,-0.041152,-0.040864
